In [31]:
import pandas as pd
import numpy as np
from scipy import stats\

df = pd.read_csv("Practical Example - Confidence Intervals.xlsx - Al Bundy.csv",skiprows=3, header=0)
df.set_index("InvoiceNo", inplace = True)

df = df.drop(columns = ["Unnamed: 0", "SalePrice", "Discount", "UnitPrice", "Size (Europe)", "Size (UK)", "Date"])
df.rename(columns = {"Unnamed: 12" : "Year"}, inplace=True)

df

,Country,ProductID,Shop,Gender,Size (US),Year,Month
InvoiceNo,,,,,,,
52389,United Kingdom,2152,UK2,Male,11.0,2014,1
52390,United States,2230,US15,Male,11.5,2014,1
52391,Canada,2160,CAN7,Male,9.5,2014,1
52392,United States,2234,US6,Female,9.5,2014,1
52393,United Kingdom,2222,UK4,Female,9.0,2014,1
...,...,...,...,...,...,...,...
65773,United Kingdom,2154,UK2,Male,9.5,2016,12
65774,United States,2181,US12,Female,12.0,2016,12
65775,Canada,2203,CAN6,Male,10.5,2016,12


Making data frames to separate year, country and gender

In [3]:
men2016_us = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Male') & (df["Country"] == "United States")]
men2016_cn = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Male') & (df["Country"] == "Canada")]
men2016_uk = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Male') & (df["Country"] == "United Kingdom")]
men2016_ger = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Male') & (df["Country"] == "Germany")]

fem2016_us = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Female') & (df["Country"] == "United States")]
fem2016_cn = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Female') & (df["Country"] == "Canada")]
fem2016_uk = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Female') & (df["Country"] == "United Kingdom")]
fem2016_ger = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Female') & (df["Country"] == "Germany")] 


Making a frequency distribution table

In [41]:
grp_month = men2016_us.groupby("Month")
freq_table = grp_month["Size (US)"].value_counts().unstack().fillna(0)

freq_table

Size (US),6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,10.5,11.0,11.5,12.0,13.0,14.0,15.0
Month,,,,,,,,,,,,,,,,
1,4.0,3.0,0.0,3.0,7.0,12.0,17.0,19.0,17.0,13.0,5.0,4.0,3.0,1.0,2.0,0.0
2,1.0,2.0,0.0,2.0,9.0,12.0,13.0,25.0,26.0,16.0,16.0,3.0,0.0,1.0,6.0,0.0
3,3.0,0.0,1.0,3.0,7.0,8.0,13.0,27.0,26.0,22.0,13.0,6.0,0.0,2.0,3.0,0.0
4,1.0,1.0,0.0,1.0,3.0,8.0,11.0,24.0,19.0,14.0,10.0,3.0,4.0,0.0,3.0,1.0
5,3.0,0.0,6.0,7.0,12.0,15.0,21.0,26.0,16.0,28.0,10.0,3.0,4.0,3.0,5.0,1.0
6,3.0,0.0,4.0,0.0,2.0,9.0,22.0,33.0,31.0,19.0,11.0,5.0,4.0,2.0,3.0,0.0
7,3.0,1.0,4.0,7.0,9.0,17.0,25.0,25.0,25.0,18.0,15.0,6.0,3.0,1.0,2.0,4.0
8,4.0,7.0,2.0,3.0,4.0,17.0,30.0,47.0,24.0,15.0,8.0,4.0,12.0,0.0,1.0,0.0
9,3.0,2.0,3.0,4.0,7.0,6.0,26.0,31.0,23.0,19.0,9.0,5.0,4.0,0.0,0.0,0.0


Calculating the mean and standar error with the frequency table

In [30]:
every_mean = freq_table.mean().round(2)
column_mean = freq_table[6.0].mean()

every_stderr = freq_table.sem().round(2)
column_stderr = freq_table[6.0].sem().round(2)

Size (US)
6.0      3.18
6.5      2.22
7.0      3.33
7.5      3.45
8.0      6.08
8.5     10.75
9.0     18.83
9.5     30.33
10.0    22.75
10.5    17.58
11.0     9.75
11.5     5.75
12.0     4.60
13.0     2.11
14.0     2.64
15.0     2.00
dtype: float64

Function to print confidence intervals

In [47]:
confidence_interval = 0.95

def conf_int_t(freq_table, confidence):
    for col in freq_table.columns:
        col_data = freq_table[col]
        mean = np.mean(col_data)
        std_error = stats.sem(col_data)
        lower_bound, upper_bound = stats.t.interval(confidence_interval, len(col_data) - 1, mean, std_error)
        mean = round(np.mean(col_data), 2)
        lower_bound = round(lower_bound, 2)
        upper_bound = round(upper_bound, 2)
        print(f"{col}: Mean={mean}, 95% Confidence Interval=({lower_bound}, {upper_bound})")
        
conf_int_t(freq_table, confidence_interval)

6.0: Mean=2.92, 95% Confidence Interval=(1.78, 4.05)
6.5: Mean=1.67, 95% Confidence Interval=(0.45, 2.89)
7.0: Mean=1.67, 95% Confidence Interval=(0.33, 3.0)
7.5: Mean=3.17, 95% Confidence Interval=(1.64, 4.7)
8.0: Mean=6.08, 95% Confidence Interval=(4.14, 8.03)
8.5: Mean=10.75, 95% Confidence Interval=(8.28, 13.22)
9.0: Mean=18.83, 95% Confidence Interval=(14.5, 23.17)
9.5: Mean=30.33, 95% Confidence Interval=(24.95, 35.72)
10.0: Mean=22.75, 95% Confidence Interval=(19.3, 26.2)
10.5: Mean=17.58, 95% Confidence Interval=(14.57, 20.59)
11.0: Mean=9.75, 95% Confidence Interval=(7.53, 11.97)
11.5: Mean=5.75, 95% Confidence Interval=(3.63, 7.87)
12.0: Mean=3.83, 95% Confidence Interval=(1.6, 6.06)
13.0: Mean=1.58, 95% Confidence Interval=(0.75, 2.42)
14.0: Mean=2.42, 95% Confidence Interval=(1.32, 3.52)
15.0: Mean=0.67, 95% Confidence Interval=(-0.12, 1.45)
